*Aprenentatge Computacional*  
- Zakaria Boudich Makran (1672001)

# Pràctica - Google Play Store Apps

En aquesta pràctica s'abordarà la creació d'un model per tal de predir el nombre de descàrregues que tindrà una aplicació mòbil, representat per l'atribut *installs*. Seguirem una sèrie de passos per obtenir el resultat, primer analitzarem les dades, les preprocessarem, s'escollirà les mètriques i els diferents models per així aconseguir el millor classificador.

## 1. Importació de llibreries 

Importem les llibreries necessàries per a la manipulació, visualització i anàlisi de dades ⚙️

In [21]:
# LLIBRERIA PER MANIPULACIÓ DE DADES
import pandas as pd

# LLIBRERIA PER TREBALLAR AMB MATRIUS MULTIDIMENSIONALS
import numpy as np

# LLIBRERIA PER DETECTAR VALORS NULS
import missingno as msno

# LLIBRERIA PER VISUALITZAR DADES
import seaborn as sns
import matplotlib.pylab as plt

# LLIBRERIA PER EINES DE PREPROCESSING
from sklearn.preprocessing import MinMaxScaler

# LLIBRERIA PER IMPORTAR DIFERENTS MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# LLIBRERIA PER ANALITZAR DIFERENTS MODELS
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score

# LLIBRERIA PER FER VALIDACIÓ DE DIFERENTS MODELS
from sklearn.model_selection import StratifiedKFold, cross_val_score, learning_curve, GridSearchCV, cross_val_predict

# LLIBRERIES PER TRANSFORMAR VARIABLES CATEGÒRIQUES A FORMATS NUMÈRICS
from category_encoders.one_hot import OneHotEncoder

# LLIBRERIA PER CREAR CÒPIES D'OBJECTES
import copy as cp

## 1. EDA (exploratory data analysis) 

En aquest apartat, realitzarem una anàlisi exploratòria de les dades, on examinarem les característiques del conjunt de dades, buscarem patrons i visualitzarem la distribució de les variables. 

En aquest cas, ens ve donat **dos conjunts de dades**:

**1. Dataset de les aplicacions (Apps):** 📲  
* Conté informació sobre les aplicacions a la Google Play Store, com el nom, la categoria, la valoració mitjana, el nombre d'instal·lacions, el preu, etc.  
* Aquest conjunt de dades ens permet entendre les característiques bàsiques de cada aplicació i el seu rendiment a la botiga.

**2. Dataset de les ressenyes (Reviews):** ⭐ 
* Conté les opinions dels usuaris sobre les aplicacions anteriors. Cada ressenya inclou la valoració que l'usuari ha donat a l'aplicació, el text de la ressenya, etc. 
* Aquest conjunt de dades és útil per estudiar el sentiment dels usuaris i com les valoracions individuals poden influir en la valoració general de l'aplicació.

**Per simplicitat**, primer tractarem el dataset d'aplicacions i després el de les ressenyes.


### 1.1. Anàlisis del dataset de les aplicacions

#### 1.1.1. Definició i significat dels atributs


In [22]:
apps_data = pd.read_csv("data/apps.csv")
apps_data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [23]:
print("El nombre de mostres que tenim a la base de dades és de", apps_data.shape[0])
print("El nombre de atributs que tenim a la base de dades és de", apps_data.shape[1])

El nombre de mostres que tenim a la base de dades és de 10841
El nombre de atributs que tenim a la base de dades és de 13


Es pot veure que disposem de 13 atributs diferents amb un total de 10841, que utilitzarem per entrenar el nostre model i predir la variable objectiu. A continuació, analitzarem detalladament cada atribut per entendre millor el seu significat i la seva possible influència en la predicció del nombre de descàrregues d'una aplicació.


In [24]:
print("Informació de cada atribut:")
print(apps_data.info())

Informació de cada atribut:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
None


La base de dades conté 13 atributs on es descriu les característiques de cada passatger. A continuació es fa una breu descripció de cada *feature*:

- **App:** Nom de l'aplicació.
- **Category:** Categoria de l'aplicació.
- **Rating:** Valoració de l'aplicació (valor de 1 a 5).
- **Reviews:** Nombre de ressenyes que té l'aplicació.
- **Size:** Mida de l'aplicació en megabytes.
- **Installs:** Nombre d'instal·lacions de l'aplicació. Correspon a la variable que volem predir, el *target*.
- **Type:** Si l'aplicació és gratuïta o de pagament.
- **Price:** El preu de l'aplicació (si és de pagament).
- **Content Rating:** Classificació per contingut (per exemple, "Tots els públics", "Adolescents", etc.).
- **Genres:** Gèneres de l'aplicació (com "Acció", "Educació", etc.).
- **Last Updated:** Data de l'última actualització de l'aplicació.
- **Current Ver:** Versió actual de l'aplicació.
- **Android Ver:** Versió d'Android compatible amb l'aplicació

**Nota 📝**\
A l'hora de descriure els tipus de dades dels atributs, no utilitzem el tipus assignat per Pandas (com objecte, enter o flotant), sinó el tipus que podem considerar segons la seva naturalesa essencial. Per exemple, l'atribut *Size*, que representa la mida de l'aplicació, es tracta com un atribut numèric, tot i que inicialment es presenta com un objecte (string) que inclou el caràcter M per indicar les unitats de mesura. Més endavant, aplicarem un procés d'escapament per eliminar aquests caràcters en i convertir els valors en nombres. 

#### 1.1.2. Valors nuls

A continuació mostrarem el nombre de NaNs per cada atribut:


In [33]:
print(apps_data.isnull().sum())

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64


Podem observar la aparició de valors NaNs en els atributs següents: *Rating*, *Current Ver*, *Android Ver*, *Type* i *Content Rating*. D'aquests atributs el més significatiu és el *Rating*, obtenint un percentatge de NaNs proper el 15%.